In [1]:
from src.Gates import QuantumGate, PauliGate, PauliExpression

In [2]:
gate_name = 'xy'
obj = PauliGate(name=gate_name)
print(obj.matrix())

[[0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]]


In [4]:
exs = 'zz+zz'
obj = PauliExpression(expression=exs)
print(obj.matrix())

[[ 2.  0.  0.  0.]
 [ 0. -2.  0.  0.]
 [ 0.  0. -2.  0.]
 [ 0.  0.  0.  2.]]
